In [1]:
from bs4 import BeautifulSoup
from fnmatch import fnmatch
from PIL import Image
from PIL import ImageFile
from pathlib import Path
from shutil import copyfile

import imagehash
import mysql.connector
import os
import re

In [2]:
# GENERAL CONFIGS
ImageFile.LOAD_TRUNCATED_IMAGES = True

# == Database config ==
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root"
)

cursor = conn.cursor()

DB_NAME = "blp_dataset"
TABLE_NAME = "products_tmp_cm"


# == Files config ==
MAIN_FOLDER = r"D:\dnmarchives\dnmarchives\cryptomarket"

In [3]:
# Creating environment
cursor.execute("CREATE DATABASE IF NOT EXISTS {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
cursor.execute("USE {}".format(DB_NAME))

cursor.execute("DROP TABLE IF EXISTS products_tmp_cm")


In [4]:
table_schema = (
    "CREATE TABLE IF NOT EXISTS products_tmp_cm ("
    "  id INT(11) AUTO_INCREMENT PRIMARY KEY NOT NULL,"
    "  name TEXT NOT NULL,"
    "  description TEXT,"
    "  market_name VARCHAR(255),"
    "  seller_name TEXT,"
    "  illegal ENUM('t', 'f') NOT NULL DEFAULT 't'"
    ") ENGINE=InnoDB")

cursor.execute(table_schema)

In [5]:
def get_content(content):
    clear_content = ""

    if content is not None:
        raw_c = content.text

        clear_content = re.sub(r'[\t\r\n]', ' ', raw_c)
        clear_content = re.sub(' +',' ', clear_content).lstrip()
        
    return clear_content

In [6]:
original_images = []

folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]

for fo in folders:
    folder = Path(f"{fo}")
    
#     try:
    for file in os.listdir(folder):            
        if "item.php_id=" in file:
            page_content = BeautifulSoup(open(Path(f"{folder}\{file}"), encoding='utf8').read())

            if page_content is not None:
                ## GET DESCRIPTION
                description = ""
                
                try:
                    desc_content = page_content.find("div", attrs={"style": "padding:20px; font-size:14px;"})

                    if desc_content is not None:
                        description = get_content(desc_content)
                except:
                    description = ""

                ## GET TITLE
                title = ""
                
                try:
                    raw_title = page_content.findAll("h3")

                    if raw_title is not None and len(raw_title) > 0:
                        title = get_content(raw_title[0])
                except:
                    title = ""

                if description != "" or title != "":
                    try:
                        # Insert product into database
                        query = "INSERT INTO products_tmp_cm (name, description, market_name, illegal) VALUES (%s, %s, %s, %s)"
                        values = (title, description, "Cryptomarket", "t")

                        cursor.execute(query, values)
                        conn.commit()

                        product_id = cursor.lastrowid

                        ## GET IMAGE
                        main_div = page_content.find("a", attrs={"target": "_blank"})

                        if main_div is not None:
                            name = main_div["href"]

                            if name is not None:
                                url_image = Path(f"{folder}\{name}")

                                try:
                                    # copy images to new directory, renaming with product id
                                    if url_image.exists():
                                        print(f"Image {product_id} exists... creating")

                                        dst_dir = Path(f"D:\images\_cm\{product_id}.jpeg")
                                        copyfile(url_image, dst_dir)
                                    else:
                                        print(f"Image {product_id} does not exists")
                                except:
                                    print(f"Image {product_id} has error. Except")
                    except:
                        print("Bad description")
#     except:
#         print(f"Error opening file into {folder}")



Image 1 does not exists
Image 2 does not exists
Image 3 exists... creating
Image 4 exists... creating
Image 5 exists... creating
Image 6 does not exists
Image 7 exists... creating
Image 8 exists... creating
Image 9 exists... creating
Image 10 exists... creating
Image 11 exists... creating
Image 12 does not exists
Image 13 exists... creating
Image 14 exists... creating
Image 15 exists... creating
Image 16 exists... creating
Image 17 exists... creating
Image 18 exists... creating
Image 19 does not exists
Image 20 exists... creating
Image 21 exists... creating
Image 22 exists... creating
Image 23 exists... creating
Image 24 exists... creating
Image 25 exists... creating
Image 26 exists... creating
Image 27 exists... creating
Image 28 exists... creating
Image 29 exists... creating
Image 30 exists... creating
Image 31 does not exists
Image 32 exists... creating
Image 33 exists... creating
Image 34 exists... creating
Image 35 exists... creating
Image 36 exists... creating
Image 37 exists... 

Image 303 exists... creating
Image 304 exists... creating
Image 305 exists... creating
Image 306 exists... creating
Image 307 exists... creating
Image 308 exists... creating
Image 309 exists... creating
Image 310 exists... creating
Image 311 exists... creating
Image 312 exists... creating
Image 313 exists... creating
Image 314 does not exists
Image 315 exists... creating
Image 316 exists... creating
Image 317 exists... creating
Image 318 exists... creating
Image 319 exists... creating
Image 320 exists... creating
Image 321 exists... creating
Image 322 does not exists
Image 323 exists... creating
Image 324 exists... creating
Image 325 exists... creating
Image 326 exists... creating
Image 327 exists... creating
Image 328 exists... creating
Image 329 exists... creating
Image 330 exists... creating
Image 331 exists... creating
Image 332 exists... creating
Image 333 exists... creating
Image 334 exists... creating
Image 335 exists... creating
Image 336 exists... creating
Image 337 exists... 

Image 609 exists... creating
Image 610 exists... creating
Image 611 exists... creating
Image 612 exists... creating
Image 613 exists... creating
Image 614 exists... creating
Image 615 exists... creating
Image 616 exists... creating
Image 617 exists... creating
Image 618 does not exists
Image 619 does not exists
Image 620 exists... creating
Image 621 exists... creating
Image 622 exists... creating
Image 623 exists... creating
Image 624 exists... creating
Image 625 exists... creating
Image 626 exists... creating
Image 627 exists... creating
Image 628 exists... creating
Image 629 exists... creating
Image 630 exists... creating
Image 631 exists... creating
Image 632 exists... creating
Image 633 exists... creating
Image 634 has error. Except
Image 635 exists... creating
Image 636 exists... creating
Image 637 exists... creating
Image 638 exists... creating
Image 639 exists... creating
Image 640 exists... creating
Image 641 exists... creating
Image 642 does not exists
Image 643 exists... crea

Image 897 does not exists
Image 898 exists... creating
Image 899 exists... creating
Image 900 exists... creating
Image 901 exists... creating
Image 902 exists... creating
Image 903 exists... creating
Image 904 does not exists
Image 905 exists... creating
Image 906 exists... creating
Image 907 exists... creating
Image 908 exists... creating
Image 909 exists... creating
Image 910 exists... creating
Image 911 exists... creating
Image 912 exists... creating
Image 913 exists... creating
Image 914 exists... creating
Image 915 exists... creating
Image 916 exists... creating
Image 917 exists... creating
Image 918 exists... creating
Image 919 does not exists
Image 920 exists... creating
Image 921 exists... creating
Image 922 exists... creating
Image 923 exists... creating
Image 924 exists... creating
Image 925 exists... creating
Image 926 exists... creating
Image 927 exists... creating
Image 928 exists... creating
Image 929 exists... creating
Image 930 exists... creating
Image 931 exists... cre

Image 1185 exists... creating
Image 1186 exists... creating
Image 1187 does not exists
Image 1188 does not exists
Image 1189 does not exists
Image 1190 does not exists
Image 1191 does not exists
Image 1192 exists... creating
Image 1193 does not exists
Image 1194 does not exists
Image 1195 does not exists
Image 1196 does not exists
Image 1197 does not exists
Image 1198 exists... creating
Image 1199 exists... creating
Image 1200 does not exists
Image 1201 exists... creating
Image 1202 does not exists
Image 1203 does not exists
Image 1204 does not exists
Image 1205 does not exists
Image 1206 does not exists
Image 1207 does not exists
Image 1208 does not exists
Image 1209 does not exists
Image 1210 exists... creating
Image 1211 does not exists
Image 1212 does not exists
Image 1213 does not exists
Image 1214 does not exists
Image 1215 exists... creating
Image 1216 exists... creating
Image 1217 does not exists
Image 1218 does not exists
Image 1219 does not exists
Image 1220 does not exists
I

...

Image 15982 has error. Except
Image 15983 has error. Except
Image 15984 has error. Except
Image 15985 has error. Except
Image 15986 has error. Except
Image 15987 has error. Except
Image 15988 has error. Except
Image 15989 has error. Except
Image 15990 has error. Except
Image 15991 has error. Except
Image 15992 has error. Except
Image 15993 has error. Except
Image 15994 has error. Except
Bad description
Image 15995 has error. Except
Image 15996 has error. Except
Image 15997 has error. Except
Image 15998 has error. Except
Image 15999 has error. Except
Image 16000 has error. Except
Image 16001 has error. Except
Image 16002 has error. Except
Image 16003 has error. Except
Image 16004 has error. Except
Image 16005 has error. Except
Image 16006 has error. Except
Image 16007 has error. Except
Image 16008 has error. Except
Image 16009 has error. Except
Image 16010 has error. Except
Image 16011 has error. Except
Image 16012 has error. Except
Image 16013 has error. Except
Image 16014 has error. E

Image 16255 has error. Except
Image 16256 has error. Except
Image 16257 has error. Except
Image 16258 has error. Except
Image 16259 has error. Except
Image 16260 has error. Except
Image 16261 has error. Except
Image 16262 has error. Except
Image 16263 has error. Except
Image 16264 has error. Except
Image 16265 has error. Except
Image 16266 has error. Except
Image 16267 has error. Except
Image 16268 has error. Except
Bad description
Image 16269 has error. Except
Image 16270 has error. Except
Image 16271 has error. Except
Image 16272 has error. Except
Image 16273 has error. Except
Image 16274 has error. Except
Image 16275 has error. Except
Image 16276 has error. Except
Image 16277 has error. Except
Image 16278 has error. Except
Image 16279 has error. Except
Image 16280 has error. Except
Image 16281 has error. Except
Image 16282 has error. Except
Image 16283 has error. Except
Image 16284 has error. Except
Image 16285 has error. Except
Image 16286 has error. Except
Image 16287 has error. E

Image 16540 has error. Except
Image 16541 has error. Except
Image 16542 has error. Except
Image 16543 has error. Except
Image 16544 has error. Except
Image 16545 has error. Except
Image 16546 has error. Except
Image 16547 has error. Except
Image 16548 has error. Except
Image 16549 has error. Except
Image 16550 has error. Except
Image 16551 has error. Except
Image 16552 has error. Except
Image 16553 has error. Except
Image 16554 has error. Except
Image 16555 has error. Except
Image 16556 has error. Except
Image 16557 has error. Except
Image 16558 has error. Except
Image 16559 has error. Except
Image 16560 has error. Except
Image 16561 has error. Except
Image 16562 has error. Except
Image 16563 has error. Except
Image 16564 has error. Except
Image 16565 has error. Except
Image 16566 has error. Except
Image 16567 has error. Except
Image 16568 has error. Except
Image 16569 has error. Except
Image 16570 has error. Except
Image 16571 has error. Except
Image 16572 has error. Except
Image 1657

Image 16816 has error. Except
Image 16817 has error. Except
Image 16818 has error. Except
Image 16819 has error. Except
Image 16820 has error. Except
Image 16821 has error. Except
Image 16822 has error. Except
Image 16823 has error. Except
Image 16824 has error. Except
Image 16825 has error. Except
Image 16826 has error. Except
Image 16827 has error. Except
Image 16828 has error. Except
Image 16829 has error. Except
Image 16830 has error. Except
Image 16831 has error. Except
Image 16832 has error. Except
Image 16833 has error. Except
Image 16834 has error. Except
Image 16835 has error. Except
Image 16836 has error. Except
Image 16837 has error. Except
Image 16838 has error. Except
Image 16839 has error. Except
Image 16840 has error. Except
Image 16841 has error. Except
Image 16842 has error. Except
Image 16843 has error. Except
Image 16844 has error. Except
Image 16845 has error. Except
Image 16846 has error. Except
Image 16847 has error. Except
Image 16848 has error. Except
Image 1684

Image 17095 has error. Except
Image 17096 has error. Except
Image 17097 has error. Except
Image 17098 has error. Except
Image 17099 has error. Except
Image 17100 has error. Except
Image 17101 has error. Except
Image 17102 has error. Except
Image 17103 has error. Except
Image 17104 has error. Except
Image 17105 has error. Except
Image 17106 has error. Except
Image 17107 has error. Except
Image 17108 has error. Except
Image 17109 has error. Except
Image 17110 has error. Except
Image 17111 has error. Except
Image 17112 has error. Except
Image 17113 has error. Except
Image 17114 has error. Except
Image 17115 has error. Except
Image 17116 has error. Except
Image 17117 has error. Except
Image 17118 has error. Except
Image 17119 has error. Except
Image 17120 has error. Except
Image 17121 has error. Except
Image 17122 has error. Except
Image 17123 has error. Except
Image 17124 has error. Except
Image 17125 has error. Except
Image 17126 has error. Except
Image 17127 has error. Except
Image 1712